<a href="https://colab.research.google.com/github/TomokiArima/Simulation2024_Homework/blob/main/%E3%82%B7%E3%83%9F%E3%83%A5%E5%B7%A5_%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#スプレッドシートを操作するようにする認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
#スプレッドシートのURLを入力
url = input("スプレッドシートのURLを入力してください: ")

try:
  sh = gc.open_by_url(url)
  print(f"スプレッドシート '{sh.title}' が正常に開かれました。")

  #スプレッドシートの最初のシートを取得
  worksheet = sh.sheet1

  start_row = input("開始行番号")  #得るセル範囲の開始行を入力
  end_row = input("終了行番号")  #終了行を入力
  cell_range = f"B{start_row}:B{end_row}" #得るセル範囲を文字列にする(B20:B60みたいなやつ)

  cell_values = worksheet.range(cell_range) #{シートを取得した奴}.range(セル範囲)でセルを含むリストを得る

  #セルオブジェクトの中から値のみをfloat型として抽出して配列に変換(.range()だけでは値以外に行・列など余計な情報を含んでいるので)
  V_call = [] #とりあえず空のリストを作る
  for i in cell_values: #ループで cell_value から値をとりだす
    try:
      V_call.append(float(i.value)) #{インデックス}.valueでi番目の要素を文字列で取り出す。それをfloat型に変換してV_callという配列に追加
    except ValueError:
      print(f"エラーが発生しました、範囲に空白や文字列が含まれているかも: {e}")

  # 配列を表示
  print(f"取得したデータ (B列 {start_row}行から{end_row}行):")
  print(V_call)

except gspread.exceptions.SpreadsheetNotFound:
  print("指定されたURLのスプレッドシートは見つかりませんでした。")
except Exception as e:
  print(f"エラーが発生しました: {e}")

スプレッドシートのURLを入力してください: https://docs.google.com/spreadsheets/d/1fy7VMw1pM5a6qfqBMF5xLG2R_Tz7CNm4Ly21xzqPuxQ/edit?gid=1906424505#gid=1906424505
スプレッドシート 'mass3_t1.0 (1).CSV' が正常に開かれました。
開始行番号117
終了行番号1025
取得したデータ (B列 117行から1025行):
[6.88, 6.56, 6.08, 5.28, 4.24, 3.04, 1.84, 0.64, -0.4, -1.44, -2.16, -2.8, -3.12, -3.12, -3.04, -2.56, -2.08, -1.44, -0.64, -0.08, 0.56, 0.96, 1.28, 1.6, 1.76, 1.6, 1.52, 1.28, 0.88, 0.4, -0.08, -0.64, -1.2, -1.76, -2.08, -2.24, -2.4, -2.24, -1.92, -1.52, -0.88, -0.08, 0.64, 1.52, 2.24, 2.88, 3.6, 3.92, 4.08, 4.0, 3.76, 3.36, 2.8, 2.16, 1.52, 0.96, 0.32, -0.24, -0.64, -0.96, -1.12, -1.2, -1.12, -0.88, -0.64, -0.32, -0.08, 0.24, 0.4, 0.56, 0.64, 0.72, 0.64, 0.56, 0.4, 0.24, 0.0, -0.32, -0.64, -0.88, -1.12, -1.2, -1.28, -1.28, -1.2, -0.96, -0.64, -0.32, 0.0, 0.4, 0.88, 1.28, 1.6, 1.84, 2.16, 2.24, 2.24, 2.24, 2.08, 1.84, 1.6, 1.28, 0.96, 0.72, 0.4, 0.24, 0.0, -0.08, -0.08, -0.08, -0.08, 0.0, 0.08, 0.08, 0.24, 0.32, 0.32, 0.32, 0.32, 0.32, 0.24, 0.08, 0.0, -0.08, 

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from IPython.display import display,clear_output

#減衰の式
def DampedOscillation (w0,k,x0,wd, t):
  x = [] #結果を格納するリストを初期化
  for ti in t: #tの各要素に対してループ、tiはi番目の配列の中身
    xi = x0 * (math.exp(-k * ti)) *(math.cos(wd * ti) + ((k/wd) * math.sin(wd * ti)))
    x.append(xi) # 結果をリストに追加
  return np.array(x) #リストをNumPy配列に変換して返す


#条件
t = np.arange(0, 10, 0.01)  #時間範囲
x0 = 7  #初期変位x0
Va = None
Vb = None

w0a = np.arange(10,40,0.1)  #w0：固有角振動数　(value=初期値、min,max=スライダー範囲、step=刻み幅、description=スライダータイトル)
ka = np.arange(0,1.0,0.1) #k：減衰係数
w0b = np.arange(10,40,0.1)  #w0：固有角振動数　(value=初期値、min,max=スライダー範囲、step=刻み幅、description=スライダータイトル)
kb = np.arange(0,1.0,0.1)

for i in w0a: #tの各要素に対してループ、tiはi番目の配列の中身
  for j in ka:
    try:
      Va = None
      wda = math.sqrt((i**2) - (j**2))  #wd：角振動数(減衰有)
      Va = DampedOscillation(i,j,x0,wda,t) #V：変位　減衰の式(DampedOscillation)に各値を代入して計算


    except ValueError:
            print("Error: wd=√(ω₀a² - ka²)が負になった可能性")



10.0 0.0
10.0 0.1
10.0 0.2
10.0 0.30000000000000004
10.0 0.4
10.0 0.5
10.0 0.6000000000000001
10.0 0.7000000000000001
10.0 0.8
10.0 0.9
10.1 0.0
10.1 0.1
10.1 0.2
10.1 0.30000000000000004
10.1 0.4
10.1 0.5
10.1 0.6000000000000001
10.1 0.7000000000000001
10.1 0.8
10.1 0.9
10.2 0.0
10.2 0.1
10.2 0.2
10.2 0.30000000000000004
10.2 0.4
10.2 0.5
10.2 0.6000000000000001
10.2 0.7000000000000001
10.2 0.8
10.2 0.9
10.299999999999999 0.0
10.299999999999999 0.1
10.299999999999999 0.2
10.299999999999999 0.30000000000000004
10.299999999999999 0.4
10.299999999999999 0.5
10.299999999999999 0.6000000000000001
10.299999999999999 0.7000000000000001
10.299999999999999 0.8
10.299999999999999 0.9
10.399999999999999 0.0
10.399999999999999 0.1
10.399999999999999 0.2
10.399999999999999 0.30000000000000004
10.399999999999999 0.4
10.399999999999999 0.5
10.399999999999999 0.6000000000000001
10.399999999999999 0.7000000000000001
10.399999999999999 0.8
10.399999999999999 0.9
10.499999999999998 0.0
10.49999999999999

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from IPython.display import display,clear_output

#減衰の式
def DampedOscillation (w0,k,x0,wd, t):
  x = [] #結果を格納するリストを初期化
  for ti in t: #tの各要素に対してループ、tiはi番目の配列の中身
    xi = x0 * (math.exp(-k * ti)) *(math.cos(wd * ti) + ((k/wd) * math.sin(wd * ti)))
    x.append(xi) # 結果をリストに追加
  return np.array(x) #リストをNumPy配列に変換して返す


#条件
t = np.arange(0, 10, 0.01)  #時間範囲
x0 = 7  #初期変位x0
Va = None
Vb = None

#スライダーの設定
w0a_slider = widgets.FloatSlider(value=5, min=1, max=100, step=0.1, description='ω₀a:')  #w0：固有角振動数　(value=初期値、min,max=スライダー範囲、step=刻み幅、description=スライダータイトル)
ka_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1, step=0.01, description='ka:') #k：減衰係数
w0b_slider = widgets.FloatSlider(value=5, min=1, max=100, step=0.1, description='ω₀b:')  #w0：固有角振動数　(value=初期値、min,max=スライダー範囲、step=刻み幅、description=スライダータイトル)
kb_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1, step=0.01, description='kb:')

#グラフを出力する所を作成
output_a = widgets.Output()
output_b = widgets.Output()
output_c = widgets.Output()

#関数：スライダーの値が変更されたときの更新処理(a)
def update_plot_a(change):
  global Va  #Vをグローバル変数として扱う
  with output_a:  #グラフを出力する所のoutputを開く
    clear_output(True)   #以前の出力を削除する
    #スライダーの値をw0,kに代入する
    w0a = w0a_slider.value  #".value"でスライダーの値を取得する
    ka = ka_slider.value

    try:
      wda = math.sqrt((w0a**2) - (ka**2))  #wd：角振動数(減衰有)
      Va = DampedOscillation(w0a,ka,x0,wda,t) #V：変位　減衰の式(DampedOscillation)に各値を代入して計算

      plt.plot(t, Va)
      plt.xlabel('t')
      plt.ylabel('Va')
      plt.title(f'w0a = {w0a:.2f}, ka = {ka:.2f}, x₀a = {x0:.2f}')
      plt.grid(True) # グリッド表示
      plt.show()

      update_plot_sum() #update_plot_sumを呼び出す(値が変わったのでグラフを更新する)

    except ValueError:
            print("Error: wd=√(ω₀a² - ka²)が負になった可能性")

#関数：スライダーの値が変更されたときの更新処理(b)
def update_plot_b(change):
  global Vb  #Vをグローバル変数として扱う
  with output_b:  #グラフを出力する所のoutputを開く
    clear_output(True)   #以前の出力を削除する
    #スライダーの値をw0,kに代入する
    w0b = w0b_slider.value  #".value"でスライダーの値を取得する
    kb = kb_slider.value

    try:
      wdb = math.sqrt((w0b**2) - (kb**2))  #wd：角振動数(減衰有)
      Vb = DampedOscillation(w0b,kb,x0,wdb,t) #V：変位　減衰の式(DampedOscillation)に各値を代入して計算

      plt.plot(t, Vb)
      plt.xlabel('t')
      plt.ylabel('Vb')
      plt.title(f'w0b = {w0b:.2f}, kb = {kb:.2f}, x₀b = {x0:.2f}')
      plt.grid(True) # グリッド表示
      plt.show()

      update_plot_sum() #update_plot_sumを呼び出す(値が変わったのでグラフを更新する)

    except ValueError:
            print("Error: wd=√(ω₀b² - kb²)が負になった可能性")

#二つのグラフを足し合わせる関数
def update_plot_sum():
  with output_c:  #グラフを出力する所のoutputを開く
    clear_output(True)   #以前の出力を削除する

    Vsum = Va + Vb

    plt.plot(t, Vsum)
    plt.xlabel('t')
    plt.ylabel('Vsum')
    plt.title(f'Vsum = Va + Vb')
    plt.grid(True)
    plt.show()

#スライダーの値の変化を検知する
w0a_slider.observe(update_plot_a,names='value')  #names='value'でスライダーの値を検知する指定。　検知された際に呼び出す関数はupdate_plot_aもしくは_b
ka_slider.observe(update_plot_a, names='value')
w0b_slider.observe(update_plot_b, names='value')
kb_slider.observe(update_plot_b, names='value')

display(w0a_slider, ka_slider, output_a) #スライダーと出力エリアをNotebook上に表示
display(w0b_slider, kb_slider, output_b)
display(output_c,)


update_plot_a(None)
update_plot_b(None)
update_plot_sum()



FloatSlider(value=5.0, description='ω₀a:', min=1.0)

FloatSlider(value=0.1, description='ka:', max=1.0, min=0.01, step=0.01)

Output()

FloatSlider(value=5.0, description='ω₀b:', min=1.0)

FloatSlider(value=0.1, description='kb:', max=1.0, min=0.01, step=0.01)

Output()

Output()